# Building an image retrieval system with deep features


## Fire up turicreate

In [1]:
import turicreate

# Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = turicreate.SFrame('./image_train_data')

# Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled turicreate, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = turicreate.load_model('http://s3.amazonaws.com/turicreate-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.24287176132202148,1.0954537391662598, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.5250879526138306, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.5660159587860107, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.129795789718628, 0.0,0.0, 0.7781944870948792, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.7178692817687988, 0.0,0.0, 0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.5781855583190918, 0.0,0.0, 0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.22067785263061523, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0,0.23753464221954346, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57378625869751, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.6589357256889343, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
knn_model = turicreate.nearest_neighbors.create(image_train, features = ['deep_features'], label = 'id')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

# Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [7]:
cat = image_train[18:19]

In [9]:
cat

id,image,label,deep_features,image_array
384,Height: 32 Width: 32,cat,"[1.0440353155136108, 0.0,0.0, 0.0, 0.0, 0.0, ...","[46.0, 45.0, 50.0, 47.0,45.0, 51.0, 45.0, 44.0, ..."


In [8]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 44.068ms     |

| Done         |         | 100         | 216.38ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.94031379506751,2
0,39777,38.46348889747021,3
0,36870,39.75596231189652,4
0,41734,39.786601414815024,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [10]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'], 'id')

In [14]:
get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.612ms      |

| Done         |         | 100         | 190.144ms    |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
384,Height: 32 Width: 32,cat,"[1.0440353155136108, 0.0,0.0, 0.0, 0.0, 0.0, ...","[46.0, 45.0, 50.0, 47.0,45.0, 51.0, 45.0, 44.0, ..."
6910,Height: 32 Width: 32,cat,"[1.5547490119934082, 0.0,0.0, 0.0, 0.0, 0.0, ...","[154.0, 133.0, 92.0,134.0, 112.0, 75.0, ..."
36870,Height: 32 Width: 32,cat,"[0.24048322439193726,0.0, 0.0, 0.0, 0.0, 0.0, ...","[16.0, 20.0, 19.0, 14.0,19.0, 17.0, 11.0, 15.0, ..."
39777,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.420721054077148, ...","[145.0, 166.0, 165.0,164.0, 185.0, 184.0, ..."
41734,Height: 32 Width: 32,cat,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 11.671506881713867, ...","[122.0, 27.0, 34.0,120.0, 24.0, 31.0, 11 ..."


Very cool results showing similar cats.

## Finding similar images to a car

In [12]:
car = image_train[8:9]

In [13]:
get_images_from_ids(knn_model.query(car))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 17.948ms     |

| Done         |         | 100         | 202.605ms    |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57378625869751, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
8977,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.1361560821533203, 0.0, ...","[186.0, 195.0, 199.0,182.0, 192.0, 198.0, ..."
24146,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.216635704040527, ...","[229.0, 231.0, 227.0,232.0, 235.0, 231.0, ..."
33261,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.15714794397354126, ...","[110.0, 118.0, 104.0,98.0, 104.0, 80.0, 92.0, ..."
44395,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,1.3475843667984009, 0.0, ...","[89.0, 95.0, 50.0, 83.0,84.0, 43.0, 69.0, 70.0, ..."


## Just for fun, let's create a lambda to find and show nearest neighbor images

In [15]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i : i + 1]))

In [16]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 4.256ms      |

| Done         |         | 100         | 222.954ms    |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57378625869751, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
8977,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,0.1361560821533203, 0.0, ...","[186.0, 195.0, 199.0,182.0, 192.0, 198.0, ..."
24146,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.216635704040527, ...","[229.0, 231.0, 227.0,232.0, 235.0, 231.0, ..."
33261,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.15714794397354126, ...","[110.0, 118.0, 104.0,98.0, 104.0, 80.0, 92.0, ..."
44395,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0,1.3475843667984009, 0.0, ...","[89.0, 95.0, 50.0, 83.0,84.0, 43.0, 69.0, 70.0, ..."


In [17]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 5.937ms      |

| Done         |         | 100         | 197.047ms    |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
593,Height: 32 Width: 32,automobile,"[1.6503394842147827, 0.0,0.0, 0.0, 0.0, 0.0, ...","[231.0, 222.0, 227.0,232.0, 217.0, 221.0, ..."
9118,Height: 32 Width: 32,automobile,"[0.9146924018859863, 0.0,0.0, 0.0, 0.0, 0.0, ...","[107.0, 117.0, 133.0,98.0, 107.0, 114.0, ..."
11000,Height: 32 Width: 32,automobile,"[0.0,0.26031047105789185, ...","[255.0, 255.0, 255.0,219.0, 222.0, 229.0, ..."
31776,Height: 32 Width: 32,automobile,"[0.7675965428352356,0.5174590349197388, 0.0, ...","[139.0, 149.0, 118.0,140.0, 148.0, 119.0, ..."
40118,Height: 32 Width: 32,automobile,"[0.38510000705718994,0.07096993923187256, ...","[157.0, 145.0, 124.0,154.0, 143.0, 121.0, ..."


In [18]:
show_neighbors(1222)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.389ms      |

| Done         |         | 100         | 182.47ms     |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
5226,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.9993773698806763, ...","[96.0, 139.0, 194.0,95.0, 138.0, 192.0, 9 ..."
17114,Height: 32 Width: 32,bird,"[0.5203297138214111, 0.0,0.0, 0.0, 0.0, 0.0, ...","[190.0, 224.0, 249.0,189.0, 223.0, 247.0, ..."
30333,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0,1.5886659622192383, 0.0, ...","[133.0, 169.0, 216.0,135.0, 172.0, 216.0, ..."
40115,Height: 32 Width: 32,bird,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.144926071166992, ...","[168.0, 190.0, 238.0,166.0, 188.0, 236.0, ..."
42193,Height: 32 Width: 32,bird,"[0.8527799248695374, 0.0,0.0, 0.34005308151245 ...","[80.0, 135.0, 182.0,80.0, 133.0, 180.0, 8 ..."


In [19]:
show_neighbors(2000)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 10.672ms     |

| Done         |         | 100         | 209.502ms    |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
19570,Height: 32 Width: 32,automobile,"[1.4622191190719604,0.366763710975647, 0.0, ...","[167.0, 167.0, 169.0,80.0, 85.0, 78.0, 59.0, ..."
30663,Height: 32 Width: 32,bird,"[1.3407803773880005, 0.0,0.0, 0.394400417804718, ...","[112.0, 116.0, 40.0,113.0, 118.0, 40.0, ..."
31776,Height: 32 Width: 32,automobile,"[0.7675965428352356,0.5174590349197388, 0.0, ...","[139.0, 149.0, 118.0,140.0, 148.0, 119.0, ..."
37030,Height: 32 Width: 32,automobile,"[0.49761223793029785,0.0, 0.0, 0.0, ...","[98.0, 103.0, 81.0, 91.0,96.0, 74.0, 71.0, 75.0, ..."
49913,Height: 32 Width: 32,automobile,"[1.20238196849823,0.3429654836654663, 0.0, ...","[188.0, 223.0, 251.0,207.0, 231.0, 247.0, ..."
